In [ ]:
# midi_entropy_eval.py
import os, math, collections
import pandas as pd
from fractions import Fraction
from music21 import converter, chord, note, stream

In [ ]:
# ---------- 유틸 ----------
def entropy_from_counts(counts, eps=1e-12):
    total = sum(counts.values())
    if total == 0:
        return 0.0
    H = 0.0
    for _, c in counts.items():
        p = (c / total) if total > 0 else 0.0
        if p > 0:
            H -= p * math.log(p + eps, 2)
    return H

def add_weight(d, key, w):
    d[key] = d.get(key, 0.0) + float(w)

def duration_quarterlength(el):
    # music21 요소의 실제 길이(quarterLength)를 안전하게 가져오기
    try:
        return float(el.quarterLength)
    except Exception:
        return 0.0

In [ ]:
# ---------- Pitch Class Entropy ----------
def pitch_class_entropy(m21score):
    """
    모든 note/rest를 순회, note만 취급.
    각 음표의 quarterLength로 시간가중 히스토그램 계산.
    """
    pc_counts = collections.defaultdict(float)
    total_time = 0.0

    for el in m21score.recurse().notes:
        if isinstance(el, note.Note):
            dur = duration_quarterlength(el)
            if dur <= 0:  # 음표 길이가 0 또는 비정상인 경우 무시
                continue
            pc = el.pitch.pitchClass  # 0~11
            add_weight(pc_counts, pc, dur)
            total_time += dur

    # 엔트로피
    H_pc = entropy_from_counts(pc_counts)
    # 최대값은 log2(12)
    H_pc_max = math.log(12, 2)
    return H_pc, H_pc_max, pc_counts, total_time


In [ ]:
# ---------- Chord Entropy ----------
def chord_entropy(m21score, min_continue_dur=0.0):
    """
    score.chordify() 결과를 시간축으로 훑어 코드 클래스 분포 계산.
    코드 라벨은 전조 불변성을 위해 'primeForm'을 문자열로 사용.
    각 코드의 quarterLength로 가중.
    """
    ch_stream = m21score.chordify()

    # 겹치는 event를 reduce한 평탄화 스트림
    flat = ch_stream.flatten().notesAndRests

    chord_counts = collections.defaultdict(float)
    total_time = 0.0

    for el in flat:
        dur = duration_quarterlength(el)
        if dur <= min_continue_dur:
            continue

        if isinstance(el, chord.Chord):
            # primeForm은 전조 불변(집합 형태만 보는 라벨)
            # 예: (0,4,7) 같은 튜플 -> 문자열로 통일
            pf = tuple(el.primeForm) if el.primeForm else ()
            label = f"PF{pf}"
            add_weight(chord_counts, label, dur)
            total_time += dur
        # rests는 코드 없음: 보통 무시(원하면 'REST'로 세도 됨)

    H_ch = entropy_from_counts(chord_counts)
    # 최대값은 log2(코드라벨의 종류 수) — 파일마다 달라져서 상대 비교 위주로 사용
    num_classes = max(1, len(chord_counts))
    H_ch_max_theoretical = math.log(num_classes, 2)

    return H_ch, H_ch_max_theoretical, chord_counts, total_time


In [ ]:
# ---------- 메인: 폴더 내 일괄 처리 ----------
def evaluate_folder(midi_dir, out_csv="midi_entropy_results.csv"):
    rows = []
    for fn in sorted(os.listdir(midi_dir)):
        if not fn.lower().endswith((".mid", ".midi")):
            continue
        path = os.path.join(midi_dir, fn)
        try:
            score = converter.parse(path)

            # Pitch Class
            H_pc, H_pc_max, pc_counts, pc_time = pitch_class_entropy(score)

            # Chord
            H_ch, H_ch_max, ch_counts, ch_time = chord_entropy(score)

            rows.append({
                "file": fn,
                "H_pitch_class": H_pc,
                "H_pitch_class_max": H_pc_max,           # ≈ 3.585
                "H_pitch_class_norm": H_pc / H_pc_max if H_pc_max > 0 else 0.0,
                "H_chord": H_ch,
                "H_chord_max_observed": H_ch_max,        # 해당 파일에서 등장한 코드라벨 수의 log2
                "H_chord_norm": H_ch / H_ch_max if H_ch_max > 0 else 0.0,
                "total_note_time_ql": pc_time,
                "total_chord_time_ql": ch_time,
                "pitch_class_counts": dict(sorted(pc_counts.items())),  # {0..11: time}
                "chord_class_counts": dict(sorted(ch_counts.items())),  # {"PF(...)": time}
            })
        except Exception as e:
            rows.append({
                "file": fn,
                "error": str(e)
            })

    df = pd.DataFrame(rows)
    df.to_csv(out_csv, index=False)
    print(f"✅ Saved: {out_csv}")
    return df


In [ ]:
if __name__ == "__main__":
    MIDI_DIR = '/Users/11gibaegitae/Deep Learning_project/Pitch_Chord_eval/COMPOSERNAME'
    evaluate_folder(MIDI_DIR, out_csv="midi_entropy_results_COMPOSERNAME.csv")